# Acquaintance immunization in SIR scale free graph : Demo

The question we are going to try to answer is :

If a vaccine against CO-VID is found tomorrow, which vaccination strategy leads to the suppression of the endemic state for a lowest immunization rate ?


We are going to use the SIR model in the context of Sclae free graph

In [1]:


from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from utils import *
from IPython.display import display





# Creation of the graph

Scale free graph is a fairly modelisation of real human interaction.

$\gamma$, the power of the scale free graph, take its value between 2 and 3. The lowest it is, most importantt is the social distancing.



In [2]:
lambda_val_x = widgets.FloatSlider(min=2.1, max=2.9, step=0.1, value=2.9, description='Gamma, the power of the law distribution')
num_samples_x = widgets.FloatLogSlider(min=3, max=6, step=1, value=4, description='Number of node in the graph')

graphs = interactive(generate_power_law_dg_sequence, {'manual': True}, lambda_val=lambda_val_x, num = num_samples_x, K0 = fixed(2/3), m=fixed(1))


display(graphs)



interactive(children=(FloatSlider(value=2.9, description='Gamma, the power of the law distribution', max=2.9, …

# SIR model: temporal approach

Here we proposed to plot the evolution of the pourcentage if Infected peaople and the Recored people with time.

On the rigth, we can observe the evoluation of the the number of Suceptible people an infected individual infects, on average, at each time. In fact, if this number is higher than 1, then the epidemic can reach en endemic state.

In [4]:
graph_x = widgets.Dropdown(
    options=['The lasgest connected compoenent', 'The full graph'],
    value='The lasgest connected compoenent',
    description='Type of graph:'
)


b_x = widgets.FloatSlider(min=0, max=1, step=0.05, value=0.25, description='Rate of transmission')
m_x = widgets.FloatSlider(min=1, max=21, step=0.5, value=10, description='Time of recovery')
indect_start_x = widgets.FloatSlider(min=0.001, max=0.05, step=0.01, value=0.05, description='Pourcentage initialisation')

plot_temporel = interactive(get_temporal_plot, {'manual': True}, graph_l = graph_x, b = b_x, m0 = m_x, Tmax=fixed(30), indect_start = indect_start_x, g1=fixed(graphs.result[0]), g2=fixed(graphs.result[1]))

display(plot_temporel)

interactive(children=(Dropdown(description='Type of graph:', options=('The lasgest connected compoenent', 'The…

### Comments : 

1- If nothing is done, we will finish  with 69% of the population removed. That’s perfectly match with the worst case scenario proposed by the NYT: 224 millions Americans can be infected by the virus vs 226 millions with this estimation.

2- With no exterior agent, the epidemic should stop a time 18, which represent 60% of the population removed

# SIR model as bond Percolation: comparaison of the different strategy of vaccination

This is a comparasion of the 3 stratgies discuss in the slide. The whole auestion is :

How to reach the hub without knowing the exact topology of the graph ?

The Acquiatance strategy uses the super useful property graph : "In average, your friend has more friend than you."


### The strategy

1. Pick K nodes at random in the (S) population

2. Look at their direct neighbors who are also (S). They will be the ones who will be immunized/ vaccinated

### The advantages

1. The strategy is purely local, requiring minimal information about randomly selected nodes and their immediate environment.

2. Because, in average, your friend has more friend than you, the neighbors of the node that you pick have, in average, a higher degree than the node

In [5]:

graph_x = widgets.Dropdown(
    options=['The lasgest connected compoenent', 'The full graph'],
    value='The lasgest connected compoenent',
    description='Type of graph:'
)
b_x = widgets.FloatSlider(min=0, max=1, step=0.05, value=0.25, description='Rate of transmission')
m_x = widgets.FloatSlider(min=1, max=21, step=0.5, value=10, description='Time of recovery')
indect_start_x = widgets.FloatSlider(min=0.001, max=0.05, step=0.01, value=0.05, description='Pourcentage initialisation')
K_x = widgets.FloatSlider(min=0.05, max=0.6, step=0.05, value=0.2, description='Init ask population at random')
N_x = widgets.IntSlider(min=1, max=20, step=1, value=5, description='Repeat train')

plot_strat = interactive(final_plots_strat, {'manual': True}, graph_l = graph_x, b = b_x, m0 = m_x, Tmax=fixed(30), 
                         indect_start = indect_start_x, g1=fixed(graphs.result[0]), g2=fixed(graphs.result[1]), 
                        N=N_x, immunization_rates = fixed([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), K=K_x)

display(plot_strat)


interactive(children=(Dropdown(description='Type of graph:', options=('The lasgest connected compoenent', 'The…


|                                                                                    | Random | Targeted |  Acquitance K = 20% |
|:----------------------------------------------------------------------------------:|:------:|:--------:|:-------------------:|
| Percentage of the population  vaccinacte in order to pass  the threshold condition |  85%`  |    5%    |         30%         |


### Comments : 

1- Random immunisation is not an efficient strategy

2- Targeted the hubs of the networks are super efficient strategy, but implies that we know the graph (which is not true)